In [1]:
# !pip install kagglehub

In [2]:
# import kagglehub

# path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

# print("Path to dataset files:", path)

In [3]:
#import Necessary Libraries
import pandas as pd
import ast
import os

In [4]:
# Data Loading
dataset_path = r"C:\Users\acer\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2"

for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

movies = pd.read_csv(r'C:\Users\acer\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_credits.csv')
credits = pd.read_csv(r'C:\Users\acer\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_movies.csv')

credits = credits[['title','genres','keywords','overview']]
movies = movies.merge(credits,on="title")

C:\Users\acer\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_credits.csv
C:\Users\acer\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_movies.csv


In [5]:
#Data Preprocessing
movies.dropna(inplace=True)

In [6]:
#Fetching only top3 cast
def convert_cast(text):
    count=0
    l = []
    for i in ast.literal_eval(text):
        if count<3:
            l.append(i['name'])
        count+=1
    return l
    
movies['cast'] = movies['cast'].apply(convert_cast)

In [7]:
#Fetching only Director name from Crew
def convert_crew(text):
    l = []
    for i in ast.literal_eval(text):
        if i['job']=='Director':
            l.append(i['name'])
    return l

movies['crew'] = movies['crew'].apply(convert_crew)

In [8]:
#Formatting genres and keywords columns
def convert(text):
    l = []
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l
    
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [9]:
#Removing the extra spaces
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1
    
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [10]:
#Text Preprocessing
movies['overview']=movies['overview'].apply(lambda x: x.split())

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new_df = movies.drop(columns=['overview','genres','keywords','cast','crew'])

from nltk.stem import WordNetLemmatizer

def stem(text):
    lemmatizer = WordNetLemmatizer()
    l = []
    for word in text:
        l.append(lemmatizer.lemmatize(word))
    return l

new_df['tags'] = new_df['tags'].apply(stem)

new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [17]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [11]:
#Applying Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()

In [12]:
#Appling cosine similarity to determine the similar movies based on tags
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)


def recommend(movie):
    index = new_df[new_df['title']==movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:5]:
        print(new_df.iloc[i[0]].title)

recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin


In [13]:
import pickle

pickle.dump(similarity,open("similarity.pkl",'wb'))
pickle.dump(new_df,open('movie_list.pkl','wb'))